In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
#!/home/ektov/bin/python35
import os
import sys
curruser = os.environ.get('USER')

sys.path.insert(0, '/opt/workspace/{user}/notebooks/support_library/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/libs/python3.5/site-packages/'.format(user=curruser))
sys.path.insert(0, '/opt/workspace/{user}/notebooks/labdata/lib/'.format(user=curruser))

sys.path.insert(0, './src')

# import tendo.singleton
import warnings
warnings.filterwarnings('ignore')

# import logging
# logging.basicConfig(filename='./__upd_runGASiteProd2SAS__.log',level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger = logging.getLogger(__name__)


import joblib
import json
from joblib import Parallel, delayed

from time import sleep
from itertools import islice
from multiprocessing import Pool, Process, JoinableQueue
from multiprocessing.pool import ThreadPool
from functools import partial
import subprocess
from threading import Thread
import time
import datetime as dt
from datetime import datetime


import re
import pandas as pd
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
from pathlib import Path
import shutil
import loader as load
from etl import ETLORA


In [3]:
ld=load.Loader(init_dsn=True, encoding='cp1251',  sep=',')
db = load.OracleDB('', '', ld._get_dsn('iskra4')) 
db.connect()
curs = db.cursor

## Read Premeditated Dummy Table From Oracle

In [27]:
table_name = 'lal_data_in'

In [28]:
query = '''
select /*+ parallel (12) */
          *
FROM {}
--where rownum < 1000
'''.format(table_name)

df = pd.read_sql(query, con=db.connection)

In [ ]:
df['CREATE_DT_DAY'] = df['CREATE_DT'].apply(lambda x: pd.to_datetime(datetime.strftime(x, format='%Y-%m-%d'), 
                                                                     format='%Y-%m-%d')
                                           )
df.head(10)

In [10]:
df.dtypes

INN                       int64
MODEL_NAME               object
HYPOTHESIS               object
CLIENT_TYPE              object
ID_SCEN                  object
CREATE_DT        datetime64[ns]
ONLY_SB                   int64
DEAL_EVENT_DT    datetime64[ns]
MODEL_TYPE               object
TARGET                    int64
CREATE_DT_DAY    datetime64[ns]
dtype: object

## Create AutoPartitioned Table 

In [30]:
str_ = 'CREATE TABLE {} ( '
for column_name, column in df.iteritems():
    if column.dtype.kind == 'O':
        if (df[column_name].str.len().max() >= 4000):
            str_+=column_name.upper() + ' ' +'CLOB, '
        else:
            if 'INN' in column_name.upper() or 'KPP' in column_name.upper():
                str_+=column_name.upper() + ' ' +'VARCHAR2(20), '
            else:
                str_+=column_name.upper() + ' ' +'VARCHAR2(800), '
    elif column.dtype.kind == 'i':
        str_+=column_name.upper() + ' ' +'NUMBER(20), '
    elif column.dtype.kind == 'f':
        if 'TARGET' in column_name.upper():
            str_+=column_name.upper() + ' ' +'NUMBER(1), '
        else:
            df[column_name] = df[column_name].fillna(0.0)
            str_+=column_name.upper() + ' ' +'FLOAT(126), '        
    elif column.dtype.kind == 'M':
        if 'CREATE_DT_DAY' in column_name.upper():
            str_+=column_name.upper() + ' ' +'DATE, '
        else:
            str_+=column_name.upper() + ' ' +'TIMESTAMP, '
    elif column.dtype.kind == 'b':
        str_+=column_name.upper() + ' ' +'BOOLEAN, '        
    else:
        None  
res=str_.strip()[:-1] + ' )' \
''' 
PARTITION BY RANGE (CREATE_DT_DAY) 
INTERVAL(NUMTODSINTERVAL(1,'DAY')) 
(PARTITION SYS_P1 VALUES LESS THAN (TO_DATE('2021-01-01', 'YYYY-MM-DD')),
 PARTITION SYS_P2 VALUES LESS THAN (TO_DATE('2021-01-02', 'YYYY-MM-DD'))
)
'''

print(res)

CREATE TABLE {} ( INN NUMBER(20), MODEL_NAME VARCHAR2(800), HYPOTHESIS VARCHAR2(800), CLIENT_TYPE VARCHAR2(800), ID_SCEN VARCHAR2(800), CREATE_DT TIMESTAMP, ONLY_SB NUMBER(20), DEAL_EVENT_DT TIMESTAMP, MODEL_TYPE VARCHAR2(800), TARGET NUMBER(20), CREATE_DT_DAY DATE ) 
PARTITION BY RANGE (CREATE_DT_DAY) 
INTERVAL(NUMTODSINTERVAL(1,'DAY')) 
(PARTITION SYS_P1 VALUES LESS THAN (TO_DATE('2021-01-01', 'YYYY-MM-DD')),
 PARTITION SYS_P2 VALUES LESS THAN (TO_DATE('2021-01-02', 'YYYY-MM-DD'))
)



## Create IN table being fetched by LAL model

In [32]:
table_name_new = 'LAL_DB_HIST_IN'

In [ ]:
sql = "DROP TABLE {}".format(table_name_new)
db.cursor.execute(sql)
db.connection.commit()
print('dropping done')

In [34]:
sql = res.format(table_name_new)

db.cursor.execute(sql)
db.connection.commit()
print('creating done')

creating done


In [35]:
ld.upload_df_or_csv(df, table_name_new, parallel=1, 
                    password='', path= None,
                    isclobe=0, isuseclobdct=0, verbose=1, njobs=1)

INSERT INTO LAL_DB_HIST_IN (INN, MODEL_NAME, HYPOTHESIS, CLIENT_TYPE, ID_SCEN, CREATE_DT, ONLY_SB, DEAL_EVENT_DT, MODEL_TYPE, TARGET, CREATE_DT_DAY) VALUES (:0,:1,:2,:3,:4,:5,:6,:7,:8,:9,:10)
Pushing data to iskra ... 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Pushed 25 lines 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished


## Check ORA Types for Table Populated from Pandas DF

In [36]:
user_tabs = 'user_tab_columns'

query = '''
select /*+ parallel (10) */
column_name, data_type||
case
when data_precision is not Null and nvl(data_scale,0)>0 then '('||data_precision||', '||data_scale||')'
when data_precision is not Null and nvl(data_scale,0)=0 then '('||data_precision||')'
when data_precision is Null and data_scale is not Null then '(*, '||data_scale||')'
when char_length>0 then '('||char_length||', '||case char_used
                                                     when 'B' then 'Byte'
                                                     when 'C' then 'Char'
                                                     else null
                                                end||')'
end||decode(nullable, 'N', 'NOT NULL') as data_type
FROM {}
where table_name = '{}'
'''.format(user_tabs, table_name_new)
df = pd.read_sql(query, con=db.connection)
df.head(20)

,COLUMN_NAME,DATA_TYPE
0,INN,NUMBER(20)
1,MODEL_NAME,"VARCHAR2(800, Byte)"
2,HYPOTHESIS,"VARCHAR2(800, Byte)"
3,CLIENT_TYPE,"VARCHAR2(800, Byte)"
4,ID_SCEN,"VARCHAR2(800, Byte)"
5,CREATE_DT,"TIMESTAMP(6)(*, 6)"
6,ONLY_SB,NUMBER(20)
7,DEAL_EVENT_DT,"TIMESTAMP(6)(*, 6)"
8,MODEL_TYPE,"VARCHAR2(800, Byte)"
9,TARGET,NUMBER(20)


## Check Updated Table

In [ ]:
query =\
'''
select * from {}
'''.format(table_name_new)
df = pd.read_sql(query, con=db.connection)
df.head(30)

## Create OUT table

In [24]:
out_table_name = 'LAL_DB_HIST_OUT'

In [25]:
sql = "DROP TABLE {}".format(out_table_name)
db.cursor.execute(sql)
db.connection.commit()
print('dropping done')

dropping done


In [26]:
sql =\
'''
CREATE TABLE {} 
( INN NUMBER(20), 
  MODEL_NAME VARCHAR2(800),
  MODEL_TYPE VARCHAR2(800),  
  ID_SCEN VARCHAR2(800), 
  HYPOTHESIS VARCHAR2(800), 
  CLIENT_TYPE VARCHAR2(800),   
  PROBA FLOAT(126), 
  CREATE_DT TIMESTAMP,
  CREATE_DT_DAY DATE ) 
  
PARTITION BY RANGE (CREATE_DT_DAY) 
INTERVAL(NUMTODSINTERVAL(1,'DAY')) 
(PARTITION SYS_P1 VALUES LESS THAN (TO_DATE('2021-01-01', 'YYYY-MM-DD')),
 PARTITION SYS_P2 VALUES LESS THAN (TO_DATE('2021-01-02', 'YYYY-MM-DD'))
)
'''.format(out_table_name)

db.cursor.execute(sql)
db.connection.commit()
print('creating done')

creating done


## Create INFO table

In [37]:
out_table_name = 'LAL_DB_HIST_INFO'

In [39]:
sql =\
'''
CREATE TABLE {} 
( MODEL_NAME VARCHAR2(800),
  MODEL_TYPE VARCHAR2(800),  
  ID_SCEN VARCHAR2(800), 
  DECILE NUMBER(20),
  DECILE_SIZE NUMBER(38),
  PROBA_MIN FLOAT(126),
  PROBA_MAX FLOAT(126),
  CUM_POSITIVE_RATIO FLOAT(126),
  CREATE_DT TIMESTAMP,
  CREATE_DT_DAY DATE ) 
  
PARTITION BY RANGE (CREATE_DT_DAY) 
INTERVAL(NUMTODSINTERVAL(1,'DAY')) 
(PARTITION SYS_P1 VALUES LESS THAN (TO_DATE('2021-01-01', 'YYYY-MM-DD')),
 PARTITION SYS_P2 VALUES LESS THAN (TO_DATE('2021-01-02', 'YYYY-MM-DD'))
)
'''.format(out_table_name)

db.cursor.execute(sql)
db.connection.commit()
print('creating done')

creating done


## Run a Query to a Specific Partition

In [ ]:
query =\
'''
select * from {}
partition for (date '2021-04-26')
'''.format(table_name_new)
df = pd.read_sql(query, con=db.connection)
df.head(20)

In [ ]:
query =\
'''
select * from {}
partition (SYS_P34399)
'''.format(table_name_new)
df = pd.read_sql(query, con=db.connection)
df.head(20)

## Get Detailed Info About Partitions

In [ ]:
query = '''
select /*+ parallel (12) */
          *
FROM ALL_TAB_PARTITIONS
WHERE TABLE_NAME LIKE '%LAL%'
'''
df = pd.read_sql(query, con=db.connection)
df.head()

## Aggregate to List All Current Partitions

In [286]:
query = '''
select /*+ parallel (12) */
          LISTAGG(PARTITION_NAME,',') WITHIN GROUP(ORDER BY TABLE_NAME) AS PART_LIST
FROM ALL_TAB_PARTITIONS
WHERE TABLE_NAME LIKE '%LAL%'
'''
df = pd.read_sql(query, con=db.connection)
df.head()

,PART_LIST
0,"SYS_P1,SYS_P2,SYS_P34412,SYS_P34413,SYS_P34414"


## Drop Partitions Older Than 2 Month

In [297]:
query = '''
BEGIN
       FOR cc IN (SELECT partition_name, high_value --
                    FROM all_tab_partitions
                   WHERE table_name = '{tbl}') LOOP
          EXECUTE IMMEDIATE
             'BEGIN
                 IF sysdate >= ADD_MONTHS(' || cc.high_value || ', 2) THEN
                    EXECUTE IMMEDIATE
                       ''ALTER TABLE {tbl} DROP PARTITION '
                      || cc.partition_name || '
                      '';
                 END IF;
              END;';
       END LOOP;
END;
'''.format(tbl=table_name_new.upper())

curs.execute(query)
db.connection.commit()

## Check Remaining Partitions

In [298]:
query = '''
select /*+ parallel (12) */
          LISTAGG(PARTITION_NAME,',') WITHIN GROUP(ORDER BY TABLE_NAME) AS PART_LIST
FROM ALL_TAB_PARTITIONS
WHERE TABLE_NAME LIKE '%LAL%'
--AND SYSDATE >= ADD_MONTHS(' || HIGH_VALUE || ',2)
'''
df = pd.read_sql(query, con=db.connection)
df.head()

,PART_LIST
0,"SYS_P34415,SYS_P34416,SYS_P34417"


## Drop Partitions Older Than 5 Days

In [ ]:
query = '''
BEGIN
       FOR cc IN (SELECT partition_name, high_value --
                    FROM all_tab_partitions
                   WHERE table_name = '{tbl}') LOOP
          EXECUTE IMMEDIATE
             'BEGIN
                 IF (sysdate - interval ''5'' DAY) <= ADD_MONTHS(' || cc.high_value || ', 0) THEN
                    EXECUTE IMMEDIATE
                       ''ALTER TABLE {tbl} DROP PARTITION '
                      || cc.partition_name || '
                      '';
                 END IF;
              END;';
       END LOOP;
END;
'''.format(tbl=table_name_new.upper())

curs.execute(query)
db.connection.commit()

In [ ]:
query = '''
select /*+ parallel (12) */
          LISTAGG(PARTITION_NAME,',') WITHIN GROUP(ORDER BY TABLE_NAME) AS PART_LIST
FROM ALL_TAB_PARTITIONS
WHERE TABLE_NAME LIKE '%LAL%'
--AND SYSDATE >= ADD_MONTHS(' || HIGH_VALUE || ',2)
'''
df = pd.read_sql(query, con=db.connection)
df.head()